In [5]:
import hashlib
import struct
import time
import random
import pprint

from requests import get

import chainparams

LOCAL_WAN_IP = get('https://api.ipify.org').text
LOCAL_PORT = 8333
PEER_IP = "77.98.116.8"
PEER_PORT = 8333


# Almost all integers are encoded in little endian. Only IP or port number are encoded big endian.
# And magic bytes looks like!

### Messages ###
# Received as streams to be deserialized
# Created as objects to be serialised and then streamed (e.g. to socket)
# Each message will have a header (including command) and a payload
# To serialize a message we combine the header and the payload
# To deserialize a message we read the header and then pass the payload to the relevant subclass for further unpacking
# Objects being (de)serialised will include: integers, strings, bools, var_ints and var_strings
# Each object will be saved into a class attribute

### Subclasses ###
# Will contain message-specific attributes and methods



#############
#  Struct   #
# Encodings #
#############
_bool = '?'
char = 's'
inv_vect = None
net_addr = None
uint8_t = 'B'
uint16_t = 'H'
uint32_t = 'I'
uint64_t = 'Q'
uchar = None
var_int = None
var_str = None


# class Serializable:
# 
#     # Base class for Serializing objects. To be expanded to streams + ... in future
# 
#     @staticmethod
#     def _to_bytes(msg, length=0, byteorder='little'):
#         if isinstance(msg, bytes):
#             return msg
#         elif isinstance(msg, int):  # or isinstance(msg, bool):
#             if length == 0:
#                 length = msg.bit_length()
#             return msg.to_bytes(length, byteorder)
#         elif isinstance(msg, str):
#             return msg.encode(encoding='UTF-8', errors='strict')
#         # TODO: add float support?
#         else:
#             return print("message of type %s not supported by _to_bytes()" % type(msg))


class Message():

    # Base Message class which can serialize payloads and generate headers

    def __init__(self, command=None, payload=None,):
        self.magic = struct.pack('>%s' % uint32_t, chainparams.mainParams.StartString)
        self.command = command
        self.command_bytes = None
        self.length = struct.pack('<%s' % uint32_t, len(payload))
        self.payload = payload
        self.checksum = None
        self.header = None
        self.serialize_header()

    def serialize_payload(self):
        # TODO: make sure all payloads are coming in in bytes!!
        self.length = struct.pack('<%s' % uint32_t, len(self.payload))

        double_hash = hashlib.sha256(hashlib.sha256(self.payload).digest()).digest()
        self.checksum = struct.pack('<4%s' % char, double_hash[:4])

    def serialize_header(self):
        self.serialize_payload()

        # serialize and pack command message
        b = self.command.encode(encoding='UTF-8', errors='strict')
        self.command_bytes = struct.pack('<12%s' % char, b)

        # Create the whole header
        self.header = b"".join([self.magic, self.command_bytes, self.length, self.checksum])

    def serialize(self):
        self.serialize_header()
        ser_msg = b"".join([self.header, self.payload])
        return ser_msg
    
    def deserialize(self):
        self.header = self.payload
        self.deserialize_header()
        return self.command, self.payload
    
    def deserialize_header(self):
        pass
    
    @staticmethod
    def to_var_int(x):
        if x < 0xFD:
            return struct.pack('<%s' % uint8_t, x)
        elif x <= 0xFFFF:
            return b"\xFD" + struct.pack('<%s' % uint16_t, x)
        elif x <= 0xFFFFFFFF:
            return b"\xFE" + struct.pack('<%s' % uint32_t, x)
        elif x <= 0xFFFFFFFFFFFFFFFF:
            return b"\xFF" + struct.pack('<%s' % uint64_t, x)
        else:
            raise RuntimeError("integer too large for type<var_int>")

    @staticmethod
    def to_var_str(x):
        s = x.encode(encoding='UTF-8', errors='strict')
        l = len(s)
        ss = struct.pack('<%s%s' % (l, char), s)
        return Message.to_var_int(l) + ss


class NetworkAddress(Serializable):

    def __init__(self, ip, services=1, port=8333):
        self.time = struct.pack(b"<I", int(time.time()))
        self.services = struct.pack('<%s' % uint64_t, services)

        if ':' in ip:
            self.ip = bytes(map(int, ip.split(':')))
        else:
            a = (b"\x00" * 10) + (b"\xFF" * 2)
            a_bytes = bytes(map(int, ip.split('.')))
            a += a_bytes
            self.ip = struct.pack('>16%s' % char, a)

        self.port = struct.pack('>%s' % uint16_t, port)
        self.address = b"".join([self.time, self.services, self.ip, self.port])
        self.addr_NT = b"".join([self.services, self.ip, self.port])

    def regenerate(self, _services, _ip, _port):
        self.__init__(services=_services, ip=_ip, port=_port)


class VersionMessage(Message):

    def __init__(self,
                 version,
                 services,
                 addr_recv,
                 addr_from,
                 user_agent,
                 start_height,
                 relay):
        self.version = version
        self.services = services
        self.timestamp = int(time.time())
        self.addr_recv = addr_recv
        self.addr_from = addr_from
        self.nonce = random.getrandbits(64)
        self.user_agent = user_agent
        self.start_height = start_height
        self.relay = relay
        self.payload = b""
        self.command = b"version"
        Message.__init__(self, command=self.command, payload=self.payload)

    def generate_nonce(self):
        self.nonce = random.getrandbits(64)
        return self.nonce

    def get_current_start_height(self):
        # TODO: implement this
        pass

    def serialize(self):
        msg = b""
        msg += struct.pack('<%s' % uint32_t, self.version)
        msg += struct.pack('<%s' % uint64_t, self.services)
        msg += struct.pack('<%s' % uint32_t, int(time.time()))
        msg += self.addr_recv.addr_NT
        msg += self.addr_from.addr_NT
        msg += struct.pack('<%s' % uint64_t, self.generate_nonce())
        msg += super().to_var_str(self.user_agent)
        msg += struct.pack('<%s' % uint32_t, self.start_height)
        msg += struct.pack('%s' % _bool, self.relay)
        self.payload = msg
        return Message.serialize(self)

    avail_services = {hex(1): 'NODE_NETWORK',
                         hex(2): 'NODE_GETUTXO',
                         hex(4): 'NODE_BLOOM',
                         hex(8): 'NODE_WITNESS',
                         hex(1024): 'NODE_NETWORK_LIMITED'
                         }


class Verack(Message):

    def __init__(self):
        self.command = b"verack"
        self.payload = b""
        Message.__init__(self, command=self.command, payload=self.payload)


class Addr(Message):

    def __init__(self):
        self.command = b"addr"
        self.payload = b""
        Message.__init__(self, command=self.command, payload=self.payload)
